In [1]:
import pandas as pd
import numpy as np
import MeCab

In [2]:
"""
input_file:インプットファイル名。拡張子付きで記載し、このノートブックと同じディレクトリに作成した./data/ディレクトリ配下にファイルを置く。
　　　　　　　　　　　　　　　　　テキストが記載されている列のヘッダーをtextとすること
encode:csvのエンコード方法を記載
"""

input_file = 'shonen_tanteidan.csv'
encode = 'sjis'

In [3]:
input_df = pd.read_csv('./data/' + input_file, encoding=encode)
result_df = input_df.copy()

In [4]:
def wakati(result_df):
    targets = result_df['text']
    target_arr = np.array([])
    target_arr2 = np.array([])
    wakati = MeCab.Tagger('-Owakati')
    meishi = MeCab.Tagger('wakati')

    for target in targets:
        #単純なわかちがき
        target_arr = np.append(target_arr, wakati.parse(target))

        #名詞のみ抽出するわかちがき
        result = meishi.parse(target)
        lines = result.split('\n')
        work_arr = np.array([])

        for line in lines:
            feature = line.split('\t')
            if len(feature) == 2: #'EOS'と''を省く
                info = feature[1].split(',')
                hinshi = info[0]
                if hinshi in ('名詞'):
                    work_arr = np.append(work_arr, info[6])
        noun_txt = ' '.join(work_arr)
        target_arr2 = np.append(target_arr2,noun_txt)

    result_df['w_text'] = target_arr
    result_df['noun_only'] = target_arr2
    return result_df

In [5]:
output_df = wakati(result_df)

output_df.to_csv('./data/' + input_file[:-4] + '_wakati.csv', encoding='utf-8-sig', index=False)